In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

import warnings
warnings.filterwarnings('ignore')

In [ ]:
data=pd.read_csv('https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv')
print(data.shape)
data.head()

(50000, 2)


,review,sentiment
0,My family and I normally do not watch local mo...,1
1,"Believe it or not, this was at one time the wo...",0
2,"After some internet surfing, I found the ""Home...",0
3,One of the most unheralded great works of anim...,1
4,"It was the Sixties, and anyone with long hair ...",0


**Sentiment count**

In [ ]:
data['sentiment'].value_counts()

,count
sentiment,
1,25000
0,25000


**Spliting the training dataset**

In [ ]:
#split dataset
#train dataset
train_reviews=data.review[:40000]
train_sentiments=data.sentiment[:40000]
#test data
test_reviews=data.review[40000:]
test_sentiments=data.sentiment[40000:]
print(train_reviews.shape,train_sentiments.shape)
print(test_reviews.shape,test_sentiments.shape)

(40000,) (40000,)
(10000,) (10000,)


**Text normalization**

In [ ]:
import nltk
nltk.download('stopwords')
#token
tokenizer=ToktokTokenizer()
#setting english stopword
stopword_list=nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


**Removing html strips and noise text**

In [ ]:
def strip_html(text):
  soup=BeautifulSoup(text, 'html.parser')
  return soup.get_text()
#removing the square brackets
def remove_between_square_brackets(text):
  return re.sub('\[[^]]*\]', '', text)

#removing the noisy text
def denoise_text(text):
  text=strip_html(text)
  text=remove_between_square_brackets(text)
  return
data['review']=data['review'].apply(denoise_text)

**Removing special characters**

In [ ]:
#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    # Handle potential non-string values
    if not isinstance(text, str):
        text = str(text)  # Convert to string if not already

    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text

#Apply function on review column
data['review']=data['review'].apply(remove_special_characters)

In [ ]:
def simple_stemmer(text):
  ps=nltk.porter.PorterStemmer()
  text=' '.join([ps.stem(word) for word in text.split()])
  return text
data['review']=data['review'].apply(simple_stemmer)

In [ ]:
stop=set(stopwords.words('english'))
print(stop)
def remove_stopwords(text, is_lowe_case=False):
  tokens=tokenizer.tokenize(text)
  tokens=[token.strip() for token in tokens]
  if is_lowe_case:
    filtered_tokens=[token for token in tokens if token not in stopword_list]
  else:
    filtered_tokens=[token for token in tokens if token.lower() not in stopword_list]
  filtered_text=' '.join(filtered_tokens)
  return filtered_text
data['review']=data['review'].apply(remove_stopwords)

{'the', 'up', 'yourselves', 'between', 'can', 'shan', 'been', 'wouldn', 'doesn', 'haven', "you've", "you'll", 'be', 'how', 'shouldn', 'all', 'won', 'theirs', 'other', 'ma', 'ours', 'didn', "doesn't", 'hadn', 'those', 'only', 'now', 'not', 'me', 'few', 'mightn', 'down', "hasn't", 're', 'what', 'my', 'he', 'they', 'hasn', 'your', 'these', 'isn', 'before', 'it', 'being', 'about', "couldn't", 'until', 'who', 'from', 'has', 'under', 'don', 'by', 'needn', 'is', "you'd", 'no', 'm', 'just', 'himself', 'to', 'in', 's', 'doing', "didn't", "weren't", 'am', 'most', 'yours', "mightn't", 'then', 'out', "shan't", 'll', 'did', 'whom', 'over', 'hers', 'do', 'again', 'couldn', "aren't", "shouldn't", 'ain', 'both', 'having', 'are', 'some', 'will', 'below', "needn't", 'y', 'but', 'aren', 'that', 'on', 'o', 'does', 'its', 'themselves', 'and', "wasn't", 'herself', 'our', 'or', 'here', 'there', 'with', 'her', 'his', 'myself', 'because', 't', 'an', 'she', "should've", "haven't", 'against', 'i', 'very', "hadn'

In [ ]:
norm_train_reviews=data.review[:40000]
norm_train_reviews[12]

'none'

In [ ]:
norm_test_reviews=data.review[40000:]
norm_test_reviews[45005]

'none'

In [ ]:
cv=CountVectorizer(min_df=0,max_df=1, binary=False, ngram_range=(1,3))
cv_train_reviews=cv.fit_transform(norm_train_reviews)
cv_test_reviews=cv.transform(norm_test_reviews)
print('BOW_CV_train:', cv_train_reviews.shape)
print('BOW_CV_train:', cv_test_reviews.shape)

InvalidParameterError: The 'min_df' parameter of CountVectorizer must be a float in the range [0.0, 1.0] or an int in the range [1, inf). Got 0 instead.